## Part One
Number of safe reports

In [1]:
tp = [7, 6, 4, 2, 1]

In [2]:
list(zip(tp[:-1], tp[1:]))

[(7, 6), (6, 4), (4, 2), (2, 1)]

In [3]:
d = [l1-l2 for l1, l2 in zip(tp[:-1], tp[1:])]
d

[1, 2, 2, 1]

In [4]:
all(map(lambda x: 1 <= x <= 3, d))

True

In [5]:
any([True, True, False])

True

In [6]:
def is_safe(levels):
    diffs = [l2 - l1 for l1, l2 in zip(levels[:-1], levels[1:])]
    inc = map(lambda d: 1 <= d <= 3, diffs)
    dec = map(lambda d: 1 <= -d <= 3, diffs)
    return any((all(inc), all(dec)))

In [7]:
test_lines = [
    "7 6 4 2 1",
    "1 2 7 8 9",
    "9 7 6 2 1",
    "1 3 2 4 5",
    "8 6 4 4 1",
    "1 3 6 7 9"
]

for line in test_lines:
    print(line, is_safe([int(l) for l in line.split()]))

7 6 4 2 1 True
1 2 7 8 9 False
9 7 6 2 1 False
1 3 2 4 5 False
8 6 4 4 1 False
1 3 6 7 9 True


In [8]:
def parse_report(line):
    return [int(l) for l in line.split()]

sum(is_safe(parse_report(line)) for line in test_lines)

2

In [9]:
with open("input.txt") as f:
    num_safe = sum(is_safe(parse_report(line)) for line in f)

In [10]:
num_safe

213

## Part Two
With Dampener

In [11]:
def is_safe_with_damp(levels):
    if is_safe(levels): return True
    for i in range(len(levels)):
        if is_safe(levels[:i] + levels[i+1:]):
            return True
    return False

In [12]:
all_levels = [parse_report(line) for line in test_lines]
all_levels

[[7, 6, 4, 2, 1],
 [1, 2, 7, 8, 9],
 [9, 7, 6, 2, 1],
 [1, 3, 2, 4, 5],
 [8, 6, 4, 4, 1],
 [1, 3, 6, 7, 9]]

In [13]:
for levels in all_levels:
    print(f"{levels} - {is_safe_with_damp(levels)}")

[7, 6, 4, 2, 1] - True
[1, 2, 7, 8, 9] - False
[9, 7, 6, 2, 1] - False
[1, 3, 2, 4, 5] - True
[8, 6, 4, 4, 1] - True
[1, 3, 6, 7, 9] - True


In [14]:
with open("input.txt") as f:
    num_safe = sum(is_safe_with_damp(parse_report(line)) for line in f)
num_safe

285

### Searching for a more optimal solution

In [15]:
%reset

In [16]:
test_levels = [
    ([7, 6, 4, 2, 1], "safe without removals"),
    ([1, 2, 7, 8, 9],  "unsafe"),
    ([9, 7, 6, 2, 1],  "unsafe"),
    ([1, 3, 2, 4, 5],  "safe after removing 2nd element 3"),
    ([8, 6, 4, 4, 1],  "safe after removing 3rd element 4"),
    ([1, 3, 6, 7, 9],  "safe without removals"),
    ([75, 77, 72, 70, 69],  "safe after removing 2nd element 77"),
    ([28, 28, 27, 26, 23],  "safe after removing 1st element 28"),
    ([20, 16, 14, 12, 10, 8, 7, 6],  "safe after removing 1st element 20"),
    ([70, 71, 73, 74, 75, 78, 79, 83], "safe after removing last element 83"),
    ([59, 56, 53, 50, 47, 47, 44, 41],  "safe after removing 5th/6th element 47")
]

In [17]:
for levels, notes in test_levels:
    diffs = [l2 - l1 for l1, l2 in zip(levels[:-1], levels[1:])]
    print(levels, " ", diffs, " ", notes)

[7, 6, 4, 2, 1]   [-1, -2, -2, -1]   safe without removals
[1, 2, 7, 8, 9]   [1, 5, 1, 1]   unsafe
[9, 7, 6, 2, 1]   [-2, -1, -4, -1]   unsafe
[1, 3, 2, 4, 5]   [2, -1, 2, 1]   safe after removing 2nd element 3
[8, 6, 4, 4, 1]   [-2, -2, 0, -3]   safe after removing 3rd element 4
[1, 3, 6, 7, 9]   [2, 3, 1, 2]   safe without removals
[75, 77, 72, 70, 69]   [2, -5, -2, -1]   safe after removing 2nd element 77
[28, 28, 27, 26, 23]   [0, -1, -1, -3]   safe after removing 1st element 28
[20, 16, 14, 12, 10, 8, 7, 6]   [-4, -2, -2, -2, -2, -1, -1]   safe after removing 1st element 20
[70, 71, 73, 74, 75, 78, 79, 83]   [1, 2, 1, 1, 3, 1, 4]   safe after removing last element 83
[59, 56, 53, 50, 47, 47, 44, 41]   [-3, -3, -3, -3, 0, -3, -3]   safe after removing 5th/6th element 47


Logic -
* If there is a single diff element with a different sign, then the element at the same index or the one after are candidates for removal.
* If there is a zero diff element, same logic as above.
* If all diffs are of the same sign, then the choose the absolute max and then same logic as above.

In [18]:
tp = [2, -1, 2, 1]


In [19]:
positives = list(map(lambda x: x > 0, tp))
num_positives = sum(positives)
print(positives, num_positives)

[True, False, True, True] 3


In [20]:
negatives = list(map(lambda x: x < 0, tp))
num_negatives = sum(negatives)
print(negatives, num_negatives)

[False, True, False, False] 1


In [21]:
zeros = list(map(lambda x: x == 0, tp))
num_zeros = sum(zeros)
print(zeros, num_zeros)

[False, False, False, False] 0


In [22]:
%reset

In [23]:
def removal_candidates(diffs):
    positives = list(map(lambda x: x > 0, diffs))
    num_positives = sum(positives)

    negatives = list(map(lambda x: x < 0, diffs))
    num_negatives = sum(negatives)

    zeros = list(map(lambda x: x == 0, diffs))
    num_zeros = sum(zeros)

    if num_positives > num_negatives and num_negatives == 1:
        neg_idx = positives.index(False)
        return neg_idx, neg_idx + 1

    if num_negatives > num_positives and num_positives == 1:
        pos_idx = negatives.index(False)
        return pos_idx, pos_idx + 1

    if num_zeros == 1:
        zero_idx = zeros.index(True)
        return zero_idx, zero_idx + 1

    if num_positives == len(positives) or num_negatives == len(negatives):
        abs_diffs = [abs(diff) for diff in diffs]
        max_abs_diff = max(abs_diffs)
        idx = abs_diffs.index(max_abs_diff)
        return idx, idx + 1

    return None

def is_safe_with_damp_optim(levels):
    def is_safe(ds):
        inc = map(lambda d: 1 <= d <= 3, ds)
        dec = map(lambda d: 1 <= -d <= 3, ds)
        return any((all(inc), all(dec)))

    diffs = [l2 - l1 for l1, l2 in zip(levels[:-1], levels[1:])]
    if is_safe(diffs): return True
    idxs = removal_candidates(diffs)
    if idxs:
        for idx in idxs:
            new_levels = levels[:idx] + levels[idx+1:]
            new_diffs = [l2 - l1 for l1, l2 in zip(new_levels[:-1], new_levels[1:])]
            if is_safe(new_diffs): return True
    return False

In [24]:
def parse_report(line):
    return [int(l) for l in line.split()]

In [25]:
with open("input.txt") as f:
    num_safe = sum(is_safe_with_damp_optim(parse_report(line)) for line in f)
    # for line in f:
    #     levels = parse_report(line)
    #     if not is_safe_with_damp_optim(levels) and is_safe_with_damp(levels):
    #         print(f"found misclassified - {levels}")

In [26]:
num_safe

285

In [27]:
def dbg(levels):
    diffs = [l2 - l1 for l1, l2 in zip(levels[:-1], levels[1:])]

    positives = list(map(lambda x: x > 0, diffs))
    num_positives = sum(positives)

    negatives = list(map(lambda x: x < 0, diffs))
    num_negatives = sum(negatives)

    zeros = list(map(lambda x: x == 0, diffs))
    num_zeros = sum(zeros)

    print(f"diffs: {diffs}")
    print(f"positives: {positives}, num_positives: {num_positives}")
    print(f"negatives: {negatives}, num_negatives: {num_negatives}")
    print(f"zeros: {zeros}, num_zeros: {num_zeros}")

    for i in range(len(levels)):
        new_levels = levels[:i] + levels[i+1:]
        if is_safe(new_levels):
            print(f"Removing [{i}]: {levels[i]} makes this safe.")
